Notebook that contains the code for the reranker RAG pipeline.

In [1]:
from src.vectorstore import connect_to_milvus, activate_collection
from src.retrievers import MilvusRerankRetriever

import os
from dotenv import load_dotenv

/Users/damir_bogdan/Library/Caches/pypoetry/virtualenvs/huberman-rag-09zSrCFX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 150874.24it/s]


In [2]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
connect_to_milvus()
collection = activate_collection()
collection.load()

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [5]:
rerank_retriever = MilvusRerankRetriever()

In [6]:
# Template

"""If you do not have the answer in the CONTEXT write "I don't know".
Do not make things up. Do not fabricate facts. """
RAG_TEMPLATE = """\
You are a Huberman Lab podcast listener. 
You will be provided a CONTEXT of the podcasts and a QUESTION.
Based on the CONTEXT answer the QUESTION.


Context:
{context}

Query:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

# Setup a chat model
chat_model = ChatOpenAI()
# Setup a retriever
retriever = MilvusRerankRetriever()

# Setup a chain
rerank_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [9]:
res = rerank_chain.invoke({"question" : "Can people be cured usingo only placebo?"})

In [10]:
res

{'response': AIMessage(content='No, people cannot be cured using only placebos. While placebos can have powerful effects on reducing symptoms and improving feelings of well-being, they cannot reduce the size of tumors or eliminate diseases like cancer. Placebos can be effective in reducing discomfort and improving certain symptoms, but they are not a cure for diseases.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 3835, 'total_tokens': 3898}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-69c3711b-1f5b-4641-b579-91d1e80ec169-0'),
 'context': [Document(page_content="So for instance, people who are told a given drug  will help them with their cancer  by reducing their symptoms of chemotherapy  or radiation therapy,  often experience reductions in the negative symptoms  of chemotherapy or radiation therapy,  reduced pain, reduced nausea,  and by consequence, improved feelings of wel